In [59]:
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from lib.Utility import exportExcelWithTimeStamp

from datetime import date, timedelta
import yfinance as yf #Alternative package if webreader does not work: pip install yfinance
import numpy as np # Fundamental package for scientific computing with Python
import joblib
import plotly.express as px
from plotly import graph_objects as go
# Train the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

# features = ['B365H','B365D', 'B365A', 'B365>2.5',	'B365<2.5']
features = ['B365D', 'B365A']

In [60]:
# data = pd.read_excel('serieA-2023-24.xlsx')
data = pd.read_excel('I1.xlsx')
# data['Cluster'] = np.where(data['FTR'] == 'H', 1, np.where(data['FTR'] == 'A', 2, 0))
data['Cluster'] = np.where(data['FTR'] == 'H', 1, 0)
data = data.dropna()
data.tail()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Cluster
375,I1,2023-06-04,20:00:00,Atalanta,Monza,5,2,H,2,0,...,-1.00,1.80,2.14,1.84,2.08,1.88,2.14,1.81,2.05,1
376,I1,2023-06-04,20:00:00,Lecce,Bologna,2,3,A,1,0,...,0.00,1.86,2.07,1.88,2.04,2.04,2.11,1.89,1.97,0
377,I1,2023-06-04,20:00:00,Milan,Verona,3,1,H,1,0,...,-0.75,1.90,2.03,1.90,2.00,1.92,2.06,1.86,2.00,1
378,I1,2023-06-04,20:00:00,Roma,Spezia,2,1,H,1,1,...,-0.75,2.04,1.89,2.04,1.88,2.11,1.93,2.02,1.84,1
379,I1,2023-06-04,20:00:00,Udinese,Juventus,0,1,A,0,0,...,0.75,2.15,1.79,2.07,1.85,2.15,1.86,2.04,1.82,0


In [61]:
x_train, x_test, y_train, y_test = train_test_split(data[features], data['Cluster'], test_size= 0.99, random_state= 1,shuffle=True)

x_train = x_train.sort_index()
x_test = x_test.sort_index()
y_train= y_train.sort_index()
y_test = y_test.sort_index()

print (f'X_train: {x_train.shape} \nX_test: {x_test.shape} \ny_train: {y_train.shape} \ny_test: {y_test.shape}')


X_train: (3, 2) 
X_test: (377, 2) 
y_train: (3,) 
y_test: (377,)


In [62]:
# Set regularization rate
reg = 0.01

# train a logistic regression model on the training set
# model = LogisticRegression(C=1/reg, solver="liblinear", random_state=42).fit(x_train, y_train)

# joblib.dump(model, "./Soccer-ml.joblib")

model = joblib.load ("./Soccer-ml.joblib")

In [63]:
df = x_test.copy()
df.to_excel("pippo.xlsx")
df['predictions'] = model.predict(x_test.sort_index())
pred_prob = model.predict_proba(x_test.sort_index())
df['y_test'] = y_test
df2 = pd.DataFrame(pred_prob)
df2.index = df.index
df = df.merge(df2, left_index=True, right_index=True)

finalDataset = data[data.index.isin(x_test.index)]
finalDataset['predictions'] = df['predictions']
finalDataset['pred_prob'] = df[1]


print(f"Accuracy score: {accuracy_score(df['y_test'], df['predictions'])}")
exportExcelWithTimeStamp(finalDataset, "Dataframe/", "df.xlsx")

Accuracy score: 0.6419098143236074


C:\Users\vitto\AppData\Local\Temp\ipykernel_34652\1535424390.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDataset['predictions'] = df['predictions']
C:\Users\vitto\AppData\Local\Temp\ipykernel_34652\1535424390.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalDataset['pred_prob'] = df[1]
